In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw3.ipynb")

---

<h1><center>SDSE Homework 3 <br><br> Simple linear regression </center></h1>

---

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pickle

# 0. Load the data

In this homework we will implement simple linear regression and apply it to four different datasets.

All of the formulas can be found in the lecture and reader.

The four datasets are contained in `hw3data.pickle`. Each dataset is a tuple with entries `x` and `y`:  `DA = (x,y)`.

In [ ]:
with open('hw3data.pickle','rb') as f:
    DA, DB, DC, DD = pickle.load(f)

# 1. Scatter plots

Display each of the four datasets using matplotlib [scatter](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html) plots. Your plot should look similar to the one below, with two rows and two columns of plots.  

<img src="f1.png" />

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=2)
ax[0,0].scatter(DA[0],DA[1],color='k',marker='.')
ax[0,1].scatter(DB[0],DB[1],color='k',marker='.')
ax[1,0].scatter(DC[0],DC[1],color='k',marker='.')
ax[1,1].scatter(DD[0],DD[1],color='k',marker='.')

# 2. Sample correlation coefficient

Implement the function `corr(D)` that takes a dataset tuple `D` and returns its sample correlation coefficient $r$.

In [ ]:
def corr(D):
    x, y = D
    muxhat = x.mean()
    muyhat = y.mean()
    return np.sum((x-muxhat)*(y-muyhat)) / np.sqrt(np.sum((x-muxhat)**2)*np.sum((y-muyhat)**2))

In [ ]:
rA = corr(DA)
rB = corr(DB)
rC = corr(DC)
rD = corr(DD)

print(f'rA={rA}')
print(f'rB={rB}')
print(f'rC={rC}')
print(f'rD={rD}')

In [ ]:
grader.check("q2")

# 3. Compute linear regresssion coefficients

Implement the function `compute_linear_regression(D)` that takes a dataset tuple `D` and returns the coefficients of the linear regression $\theta_0$ and $\theta_1$.

In [ ]:
def compute_linear_regression(D):
    x, y = D
    muxhat = x.mean()
    muyhat = y.mean()
    theta1 = np.sum((x-muxhat)*(y-muyhat)) / np.sum((x-muxhat)**2)
    theta0 = muyhat - muxhat*theta1
    return theta0, theta1

In [ ]:
thetaA = compute_linear_regression(DA)
thetaB = compute_linear_regression(DB)
thetaC = compute_linear_regression(DC)
thetaD = compute_linear_regression(DD)

print(f'A: {thetaA}')
print(f'B: {thetaB}')
print(f'C: {thetaC}')
print(f'D: {thetaD}')

In [ ]:
grader.check("q3")

# 4. Evaluate linear regresssion prediction

Implement the function `eval_linreg(theta,D)` that takes parameters $\theta=(\theta_0,\theta_1$ and the dataset `D=(x,y)` and returns the array of predictions $\hat{y}$ for the training data:
$$\hat{y} = \theta_0 + x \theta_1$$

In [ ]:
def eval_linreg(theta,D):
    theta0, theta1 = theta
    x, y = D
    return theta0 + x*theta1

In [ ]:
yhatA = eval_linreg(thetaA,DA)
yhatB = eval_linreg(thetaB,DB)
yhatC = eval_linreg(thetaC,DC)
yhatD = eval_linreg(thetaD,DD)

print(f'yhatA[:20:3]={yhatA[:20:3]}')
print(f'yhatB[:20:3]={yhatB[:20:3]}')
print(f'yhatC[:20:3]={yhatC[:20:3]}')
print(f'yhatD[:20:3]={yhatD[:20:3]}')

In [ ]:
grader.check("q4")

# 5. Plot linear regresssion prediction

Overlay the linear regression lines on each one of the scatter plots. Your plot should look like this:

<img src="f2.png" />

In [ ]:
fig, ax = plt.subplots(figsize=(12,8),nrows=2,ncols=2)
ax[0,0].scatter(DA[0],DA[1],color='k',marker='.')
ax[0,0].plot(DA[0],eval_linreg(thetaA,DA))
ax[0,1].scatter(DB[0],DB[1],color='k',marker='.')
ax[0,1].plot(DB[0],eval_linreg(thetaB,DB))
ax[1,0].scatter(DC[0],DC[1],color='k',marker='.')
ax[1,0].plot(DC[0],eval_linreg(thetaC,DC))
ax[1,1].scatter(DD[0],DD[1],color='k',marker='.')
ax[1,1].plot(DD[0],eval_linreg(thetaD,DD))

# 6. Coefficient of determinination

Implement the function `eval_Rsquared(theta,D)` that takes parameters $\theta=(\theta_0,\theta_1)$ and the dataset `D` and returns the coefficient of determination $R^2$. Verify the identity $R^2=r^2$


In [ ]:
def eval_Rsquared(theta,D):
    x, y = D
    yhat = eval_linreg(theta,D)
    muyhat = y.mean()
    return 1 - np.sum((y-yhat)**2)/np.sum((y-muyhat)**2)

In [ ]:
RsquaredA = eval_Rsquared(thetaA,DA)
RsquaredB = eval_Rsquared(thetaB,DB)
RsquaredC = eval_Rsquared(thetaC,DC)
RsquaredD = eval_Rsquared(thetaD,DD)

print(f'RsquaredA={RsquaredA}')
print(f'RsquaredB={RsquaredB}')
print(f'RsquaredC={RsquaredC}')
print(f'RsquaredD={RsquaredD}')

# 7. Estimate $\sigma^2$

Implement the function `estimate_sigma2(theta,D)` that takes parameters $\theta=(\theta_0,\theta_1)$ and the dataset `D` and returns an unbiased estimate of the variance $\sigma^2$ of the uncertainty values $\varepsilon_i$ in $y_i=\theta_0^* + x_i\theta_1^*  + \varepsilon_i$.

$$\hat\sigma^2  = \frac{1}{N-2} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2$$


In [ ]:
def estimate_sigma2(theta,D):
    x, y = D
    N = x.shape[0]
    yhat = eval_linreg(theta,D)
    return np.sum((y-yhat)**2)/(N-2)

In [ ]:
sigma2A = estimate_sigma2(thetaA,DA)
sigma2B = estimate_sigma2(thetaB,DB)
sigma2C = estimate_sigma2(thetaC,DC)
sigma2D = estimate_sigma2(thetaD,DD)

print(f'sigma2A={sigma2A}')
print(f'sigma2B={sigma2B}')
print(f'sigma2C={sigma2C}')
print(f'sigma2D={sigma2D}')

In [ ]:
grader.check("q7")

# 8. Uncertainty in the linear regression coefficients

Implement the function `estimate_sigma2(theta,D)` that takes parameters $\theta=(\theta_0,\theta_1)$ and the dataset `D` and returns the variance associated with the linear regression parameter estimates $Var[\hat\Theta_0]$ and $Var[\hat\Theta_1]$.

In [ ]:
def compute_var_param(theta,D):
    x, y = D
    N = x.shape[0]
    muxhat = x.mean()
    yhat = eval_linreg(theta,D)
    sigma2hat = estimate_sigma2(theta,D)
    sigmaX2hat = np.sum((x-muxhat)**2)/(N-1)
    var0 = sigma2hat/N + (sigma2hat*muxhat**2)/(N-1)/sigmaX2hat
    var1 = sigma2hat/(N-1)/sigmaX2hat
    return var0, var1

In [ ]:
varA = compute_var_param(thetaA,DA)
varB = compute_var_param(thetaB,DB)
varC = compute_var_param(thetaC,DC)
varD = compute_var_param(thetaD,DD)

print(f'A: {varA}')
print(f'B: {varB}')
print(f'C: {varC}')
print(f'D: {varD}')

In [ ]:
grader.check("q8")

# 9. Confidence intervals and hypothesis tests

Implement the function `compute_rho(theta,D,gamma)` that takes parameters $\theta=(\theta_0,\theta_1)$, the dataset `D`, and a confidence level $\gamma$, and returns the radiuses $\rho_0$ and $\rho_1$ of the $\gamma$-confidence intervals for $\theta^*_0$ and $\theta^*_1$ respectively.

In [ ]:
def compute_rho(theta,D,gamma):
    var0, var1 = compute_var_param(theta,D)
    rho0 = np.sqrt(var0)*abs(stats.norm().ppf((1-gamma)/2))
    rho1 = np.sqrt(var1)*abs(stats.norm().ppf((1-gamma)/2))
    return rho0, rho1

In [ ]:
gamma = 0.98
rhoA = compute_rho(thetaA,DA,gamma)
rhoB = compute_rho(thetaB,DB,gamma)
rhoC = compute_rho(thetaC,DC,gamma)
rhoD = compute_rho(thetaD,DD,gamma)

print(f'rhoA={rhoA}')
print(f'rhoB={rhoB}')
print(f'rhoC={rhoC}')
print(f'rhoD={rhoD}')

In [ ]:
grader.check("q9")

# 10. Negligible parameters
For which of the slope parameters `thetaA[1]`, `thetaB[1]`, `thetaC[1]`, or `thetaD[1]` is the null hypothesis
$$ H_0 : \theta^*_1=0$$
**not** rejected in favor of
$$ H_1 : \theta^*_1\neq0$$
with $\alpha=0.05$ significance level?

Provide your answer as a string from `{'A','B','C','D'}`

In [ ]:
def hypothesis_test(theta,D,alpha):
    _, var1 = compute_var_param(theta,D)
    p = 2*(1-stats.norm().cdf(abs(theta[1]/np.sqrt(var1))))
    return p<alpha
alpha = 0.05
print(hypothesis_test(thetaA,DA,alpha))
print(hypothesis_test(thetaB,DB,alpha))
print(hypothesis_test(thetaC,DC,alpha))
print(hypothesis_test(thetaD,DD,alpha))
answer = 'D'

# 11. Nonlinear feature vector

The dataset `DA` looks like it could be modeled as an exponential decay. Let's use this insight to see whether we can improve the regression model for this dataset. Define a feture vector `phi` with
$$\phi(x) = e^{\lambda x}$$
Using trial-and-error, find a value for $\lambda$ that achieves an $R^2$ of 0.9 or more.

In [ ]:
lbda = -0.1
x, y = DA
phi = np.exp(lbda*x)
DA2 = (phi,y)
thetaA2 = compute_linear_regression(DA2)
RsquaredA2 = eval_Rsquared(thetaA2,DA2)
print(RsquaredA2)

fig, ax = plt.subplots()
ax.scatter(DA[0],DA[1],color='k',marker='.')
ax.plot(DA[0],eval_linreg(thetaA,DA),label='$R^2$={:.3f}'.format(RsquaredA))
ax.plot(DA[0],eval_linreg(thetaA2,DA2),'-',label='$R^2$={:.3f}'.format(RsquaredA2))
ax.legend(fontsize=14)

In [ ]:
grader.check("q11")

# 12. Prediction uncertainty


Implement the function `compute_var_prediction(theta,D,gamma,xi)` that takes parameters $\theta$, the dataset `D`, and a data sample $x_i$, and returns the variance in the prediction $\hat{y}_i$ corresponding to $x_i$.

In [ ]:
def compute_var_prediction(theta,D,xi):
    x, y = D
    N = x.shape[0]
    muxhat = x.mean()
    sigma2hat = estimate_sigma2(theta,D)
    return sigma2hat*(1/N + ((xi-muxhat)**2)/np.sum((x-muxhat)**2))

In [ ]:
varYA = compute_var_prediction(thetaA,DA,DA[0])
stddevYA = np.sqrt(varYA)

varYB = compute_var_prediction(thetaB,DB,DB[0])
stddevYB = np.sqrt(varYB)

varYC = compute_var_prediction(thetaC,DC,DC[0])
stddevYC = np.sqrt(varYC)

varYD = compute_var_prediction(thetaD,DD,DD[0])
stddevYD = np.sqrt(varYD)

print(f'stddevYA={stddevYA[:20:3]}')
print(f'stddevYB={stddevYB[:20:3]}')
print(f'stddevYC={stddevYC[:20:3]}')
print(f'stddevYD={stddevYD[:20:3]}')

In [ ]:
grader.check("q12")

# 13. Plot the prediction with uncertainty bounds

Repeat the scatter and prediction plot, but this time overlay an prediction uncertainty band of plus/minus two standard deviations. Your plot should look something like this:

<img src="f3.png" />


In [ ]:
fig, ax = plt.subplots(figsize=(12,8),nrows=2,ncols=2)

a = ax[0,0]
yhatA = eval_linreg(thetaA,DA)
a.scatter(DA[0],DA[1],color='k',marker='.')
a.plot(DA[0],yhatA)
a.fill_between(DA[0],yhatA-2*stddevYA,yhatA+2*stddevYA,alpha=0.5)

a = ax[0,1]
yhatB = eval_linreg(thetaB,DB)
a.scatter(DB[0],DB[1],color='k',marker='.')
a.plot(DB[0],yhatB)
a.fill_between(DB[0],yhatB-2*stddevYB,yhatB+2*stddevYB,alpha=0.5)

a = ax[1,0]
yhatC = eval_linreg(thetaC,DC)
a.scatter(DC[0],DC[1],color='k',marker='.')
a.plot(DC[0],yhatC)
a.fill_between(DC[0],yhatC-2*stddevYC,yhatC+2*stddevYC,alpha=0.5)

a = ax[1,1]
yhatD = eval_linreg(thetaD,DD)
a.scatter(DD[0],DD[1],color='k',marker='.')
a.plot(DD[0],yhatD)
a.fill_between(DD[0],yhatD-2*stddevYD,yhatD+2*stddevYD,alpha=0.5)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)